# Mean-Variance Model

## Exercise 2. 
### Task 0: Cleaning Data and Importing Libraries: 

In [4]:
# pip install openpyxl  #install this library to open excel file

In [5]:
# pip install gurobipy #install this library to optimize the problems

In [27]:
#import required libraries: 
import gurobipy as gb
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [28]:
#load the data
df=pd.read_excel("/workspaces/FOAM/data/data.xlsx", sheet_name="Returns S&P Mib 30 ", index_col=0, parse_dates=True)
df.head(2)

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16,A17,A18,A19,A20
Dates,,,,,,,,,,,,,,,,,,,,
2004-01-01,0.056941,0.055085,-0.005061,-0.001376,0.038502,0.004092,-0.003996,-0.023231,0.036722,-0.018517,0.023003,0.008654,-0.025495,-0.028095,0.134228,-0.035847,0.039152,-0.031417,0.027356,-0.027555
2004-02-01,-0.025385,-0.097992,-0.083576,-0.046853,-0.041082,0.020377,-0.064916,-0.025405,-0.035421,-0.140655,-0.066209,-0.094376,-0.037007,-0.009937,0.064431,-0.106229,-0.009411,-0.077847,0.004931,-0.083825


In [29]:
# we can drop the index name, and change the date format appropriately
df.index.name=None
df.index = df.index.strftime('%b-%Y')
df.head(2)

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16,A17,A18,A19,A20
Jan-2004,0.056941,0.055085,-0.005061,-0.001376,0.038502,0.004092,-0.003996,-0.023231,0.036722,-0.018517,0.023003,0.008654,-0.025495,-0.028095,0.134228,-0.035847,0.039152,-0.031417,0.027356,-0.027555
Feb-2004,-0.025385,-0.097992,-0.083576,-0.046853,-0.041082,0.020377,-0.064916,-0.025405,-0.035421,-0.140655,-0.066209,-0.094376,-0.037007,-0.009937,0.064431,-0.106229,-0.009411,-0.077847,0.004931,-0.083825


In [30]:
# since the stocks are given in the range from A1 to A20, we prepared dictionary to replace them with their respective names
stocks = [
    "AL", "AGL", "AUTO", "NTV", "BFI", "BIN", "BPM", "BPVN", "BNL", "BPU", "BUL", "CAP", 
    "EDN", "ENEL", "ENI", "FWB", "F", "FNC", "G", "ES"
]

stock_dict = {f"A{i+1}": stock for i, stock in enumerate(stocks)}

# now we can replace them
df = df.rename(columns=stock_dict)
df.head(2)

,AL,AGL,AUTO,NTV,BFI,BIN,BPM,BPVN,BNL,BPU,BUL,CAP,EDN,ENEL,ENI,FWB,F,FNC,G,ES
Jan-2004,0.056941,0.055085,-0.005061,-0.001376,0.038502,0.004092,-0.003996,-0.023231,0.036722,-0.018517,0.023003,0.008654,-0.025495,-0.028095,0.134228,-0.035847,0.039152,-0.031417,0.027356,-0.027555
Feb-2004,-0.025385,-0.097992,-0.083576,-0.046853,-0.041082,0.020377,-0.064916,-0.025405,-0.035421,-0.140655,-0.066209,-0.094376,-0.037007,-0.009937,0.064431,-0.106229,-0.009411,-0.077847,0.004931,-0.083825


## Exercise 2.1 expected_return and variance and covariance matrix

In [31]:
#expected returns are found as follows, since the data frame is already about the returns we can just take the mean for an asset for the whole period. 
expected_returns = df[stocks].mean()
expected_returns.head(2)

AL     0.001935
AGL    0.028860
dtype: float64

In [32]:
# Compute variance and covariance matrix
covariance_matrix = df[stocks].cov()
covariance_matrix.head(2)

,AL,AGL,AUTO,NTV,BFI,BIN,BPM,BPVN,BNL,BPU,BUL,CAP,EDN,ENEL,ENI,FWB,F,FNC,G,ES
AL,0.005113,0.000429,0.001942,0.000599,0.000211,0.000246,0.000022,-0.000506,0.000008,0.001111,0.000096,0.001030,0.001152,0.000063,0.000536,0.001054,0.000214,0.003113,0.001102,0.000578
AGL,0.000429,0.006196,0.002060,0.001682,0.001280,0.001742,0.002319,0.001718,0.002281,0.002289,0.001349,0.001539,0.001463,0.001753,0.001833,0.001800,0.001016,0.002068,0.001396,0.000841


### Model Implementation

In [33]:
# Create model
m = gb.Model()

In [34]:
# Add a variable for each stock
x = pd.Series(m.addVars(stocks, name='X'), index=stocks)
m.update()
x.head(2)

AL      <gurobi.Var X[AL]>
AGL    <gurobi.Var X[AGL]>
dtype: object

In [36]:
# Compute portfolio variance and return
portfolio_variance = covariance_matrix.dot(x).dot(x)
portfolio_return = expected_returns.dot(x)

In [38]:
# Add objective function to model
m.setObjective(expected_returns, sense=gb.GRB.MAXIMIZE)

In [39]:
# Add budget constraint
m.addConstr(x.sum() == 1);

In [47]:
# Add portfolio variance constraint
sigma_squared = 2.02  #target variance value
portfolio_variance_constraint = m.addConstr(portfolio_variance == sigma_squared)

In [48]:
m.optimize()

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (linux64 - "Ubuntu 20.04.6 LTS")

CPU model: Intel(R) Xeon(R) Platinum 8370C CPU @ 2.80GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 1 rows, 20 columns and 20 nonzeros
Model fingerprint: 0x339529d6
Model has 210 quadratic objective terms
Model has 3 quadratic constraints
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [2e-05, 6e-03]
  Objective range  [0e+00, 0e+00]
  QObjective range [3e-05, 1e-02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
  QRHS range       [2e-02, 2e+00]
Presolve time: 0.01s

Barrier solved model in 0 iterations and 0.01 seconds (0.00 work units)
Model is infeasible or unbounded


In [45]:
if m.status == GRB.OPTIMAL:
    print("\nOptimal Portfolio Solution:")
    print("-" * 30)
    print(f"{'Asset':<10}{'Weight':>10}")
    print("-" * 30)
    for i in range(n_assets):
        print(f"{'Asset ' + str(i+1):<10}{x[i].X:>10.4f}")
    print("-" * 30)
    print(f"Expected Portfolio Return: {m.ObjVal:.4f}")
    variance = sum(covariance_matrix[i][j] * x[i].X * x[j].X for i in range(n_assets) for j in range(n_assets))
    print(f"Portfolio Variance: {variance:.6f}")
elif m.status == GRB.INFEASIBLE:
    print("Model is infeasible. Run diagnostics for more details.")


In [46]:
m.Params.LogToConsole = 1  # Display solver progress
m.Params.OutputFlag = 1    # Turn solver output ON


Set parameter LogToConsole to value 1
Set parameter OutputFlag to value 1


In [37]:
x

AL        <gurobi.Var X[AL]>
AGL      <gurobi.Var X[AGL]>
AUTO    <gurobi.Var X[AUTO]>
NTV      <gurobi.Var X[NTV]>
BFI      <gurobi.Var X[BFI]>
BIN      <gurobi.Var X[BIN]>
BPM      <gurobi.Var X[BPM]>
BPVN    <gurobi.Var X[BPVN]>
BNL      <gurobi.Var X[BNL]>
BPU      <gurobi.Var X[BPU]>
BUL      <gurobi.Var X[BUL]>
CAP      <gurobi.Var X[CAP]>
EDN      <gurobi.Var X[EDN]>
ENEL    <gurobi.Var X[ENEL]>
ENI      <gurobi.Var X[ENI]>
FWB      <gurobi.Var X[FWB]>
F          <gurobi.Var X[F]>
FNC      <gurobi.Var X[FNC]>
G          <gurobi.Var X[G]>
ES        <gurobi.Var X[ES]>
dtype: object